In [22]:
###############################################################################################
'''                          import the bookstores you need                                 '''
###############################################################################################
from math import pi,sin,cos,sqrt,acos 
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt 
import numpy as np
import random
###############################################################################################
'''                      solve the differential equations: RGE                              '''
###############################################################################################
#we define differential equations to be solved
dy1 = lambda s,y1,y2,y3: (s**(-1))*(-(41/10)/(2*pi)-(1/(8*pi**2))*((199/50)/y1+(27/10)/y2+(44/5)/y3))
dy2 = lambda s,y1,y2,y3: (s**(-1))*(-(-19/6)/(2*pi)-(1/(8*pi**2))*((9/10)/y1+(35/6)/y2+(12)/y3))
dy3 = lambda s,y1,y2,y3: (s**(-1))*(-(-7)/(2*pi)-(1/(8*pi**2))*((11/10)/y1+(9/2)/y2+(-26)/y3))                               

sqr_s0=91.1876  #initial energy 
sqr_sn=11*sqr_s0  #final energy

h = 91.1876  #step

y1 = 59.0116 #initial conditions
y2= 29.5874
y3= 8.4388

n=int((sqr_sn-sqr_s0)/h) #step number

#make arrangements 
energy = [] 
alpha1_inv = [] 
alpha2_inv = [] 
alpha3_inv = [] 

alpha1_inv.append(y1)
alpha2_inv.append(y2)
alpha3_inv.append(y3)


"""
Solving differential equations by the Runge-Kutta 4 method
"""
s=sqr_s0
for i in range(n+1):
    K1 = h*dy1(s, y1,y2,y3)
    L1 = h*dy2(s, y1,y2,y3)
    N1 = h*dy3(s, y1,y2,y3)

    K2 = h*dy1(s + h/2, y1 + K1/2,y2 + L1/2,y3 + N1/2)
    L2 = h*dy2(s + h/2, y1 + K1/2,y2 + L1/2,y3 + N1/2)
    N2 = h*dy3(s + h/2, y1 + K1/2,y2 + L1/2,y3 + N1/2)

    K3 = h*dy1(s + h/2, y1 + K2/2,y2 + K2/2,y3 + K2/2)
    L3 = h*dy2(s + h/2, y1 + K2/2,y2 + K2/2,y3 + K2/2)
    N3 = h*dy3(s + h/2, y1 + K2/2,y2 + K2/2,y3 + K2/2)

    K4 = h*dy1(s +h, y1 + K3,y2 + K3,y3 + K3)
    L4 = h*dy2(s +h, y1 + K3,y2 + K3,y3 + K3)
    N4 = h*dy3(s +h, y1 + K3,y2 + K3,y3 + K3)

    y1 += (K1 + 2*K2 + 2*K3 + K4)/6
    y2 += (L1 + 2*L2 + 2*L3 + L4)/6
    y3 += (N1 + 2*N2 + 2*N3 + N4)/6

    alpha1_inv.append(y1)
    alpha2_inv.append(y2)
    alpha3_inv.append(y3)
    energy.append(s)      #save the mass center energy in an vector
    s += h
###############################################################################################
'''                          computed the Dirac matrices                                    '''
###############################################################################################
"""
As an exercise, we have calculated the Dirac matrices 
by defining equation 9. 

This part of the code can be omitted and the Dirac matrices 
can be defined in an array.
"""

ALP1=np.array([[0,0,0,1],    #define alpha matrices
               [0,0,1,0],
               [0,1,0,0],
               [1,0,0,0]])
ALP2=np.array([[0,0,0,complex(0,-1)],
               [0,0,complex(0,1),0],
               [0,complex(0,-1),0,0],
               [complex(0,1),0,0,0]])
ALP3=np.array([[0,0,1,0],
               [0,0,0,-1],
               [1,0,0,0],
               [0,-1,0,0]])

GM0=np.array([[1,0,0,0], 
              [0,1,0,0],
              [0,0,-1,0],
              [0,0,0,-1]])

GM1=np.zeros_like(GM0)
for i in range(len(GM0)):
    for j in range(len(GM0)):
        for k in range(len(GM0)):
            GM1[i,j]+=(GM0[i,k])*(ALP1[k,j]) #gamma 1

GM2=np.zeros_like(ALP2)
for i in range(len(GM0)):
    for j in range(len(GM0)):
        for k in range(len(GM0)):
            GM2[i,j]+=(GM0[i,k])*(ALP2[k,j]) #gamma 2

GM3=np.zeros_like(GM0)
for i in range(len(GM0)):
    for j in range(len(GM0)):
        for k in range(len(GM0)):
            GM3[i,j]+=(GM0[i,k])*(ALP3[k,j]) #gamma 3

x1=np.zeros_like(ALP2)
x2=np.zeros_like(x1)
x3=np.zeros_like(x2)
for i in range(len(GM0)):
    for j in range(len(GM0)):
        for k in range(len(GM0)):
            x1[i,j]+=(GM0[i,k])*(GM1[k,j])
for i in range(len(GM0)):
    for j in range(len(GM0)):
        for k in range(len(GM0)):
            x2[i,j]+=(x1[i,k])*(GM2[k,j])
for i in range(len(GM0)):
    for j in range(len(GM0)):
        for k in range(len(GM0)):
            x3[i,j]+=(x2[i,k])*(GM3[k,j])
GM5=complex(0,1)*x3                       #gamma 5  

GM=np.array([GM0,GM1,GM2,GM3]) #save the gamma arrays in an array
###############################################################################################
'''              Create a function that computes the invariant amplitude                    '''
###############################################################################################
"""
(p0,p1,p2,p3) is a four-vector of the electron  incoming 
(p_0,p_1,p_2,p_3) is a four-vector of the electron outgoing
(k0,k1,k2,k3) is a four-vector of the electron anti-neutrino incoming
(k_0,k_1,k_2,k_3) is a four-vector of the electron anti-neutrino outgoing
me is the electron mass 
ps is the p-slash matrix 
ks is the k-slash matrix
p_s is the p'-slash matrix 
k_s is the k'-slash matrix
"""
def amplitude(p1,p2,p3,k1,k2,k3,me): #invariant amplitude function 
    p0=(me**2+p1**2+p2**2+p3**2)**0.5 #energy in natural units 
    k0=(k1**2+k2**2+k3**2)**0.5
    p_1=p1                           #use energy conservation
    p_2=p2
    p_3=p3
    k_1=k1
    k_2=k2
    k_3=k3
    p_0=p0
    k_0=k0
    p=np.array([p0,p1,p2,p3]) #save in a array
    k=np.array([k0,k1,k2,k3]) 
    k_=np.array([k_0,k_1,k_2,k_3])  
    p_=np.array([p_0,p_1,p_2,p_3])  
    
    X=np.zeros_like(GM[2])  #computed p-slash
    for i in range(1,3+1):
        x=GM[i]*p[i]
        X+=x
        i=i+1
    ps=GM[0]*p[0]-X     

    X=np.zeros_like(GM[2])  #computed k-slash
    for i in range(1,3+1):
        x=GM[i]*k[i]
        X+=x
        i=i+1
    ks=GM[0]*k[0]-X     

    X=np.zeros_like(GM[2])  #computed p'-slash
    for i in range(1,3+1):
        x=GM[i]*p_[i]
        X+=x
        i=i+1
    p_s=GM[0]*p_[0]-X   

    X=np.zeros_like(GM[2]) #computed k'-slash
    for i in range(1,3+1):
        x=GM[i]*k_[i]
        X+=x
        i=i+1
    k_s=GM[0]*k_[0]-X   

    A=np.identity(4)-GM5  #1-gamma5
    
    #in the next part, we calculated the traces for resolve the invariant amplitude
    def N1(a,b):
        N1=((((GM[a]@A)@(ps+me))@GM[b])@A)@ks 
        return N1
    def N2(a,b):
        N2=((((GM[a]@A)@k_s)@GM[b])@(A))@(p_s+me)
        return N2

    tr0=np.trace(N1(0,0))*np.trace(N2(0,0))

    tr1=0
    for a in range(1,3+1):
        AI=np.trace(N1(a,0))*np.trace(N2(a,0))
        tr1+=AI

    tr2=0
    for b in range(1,3+1):
        AI=np.trace(N1(0,b))*np.trace(N2(0,b))
        tr2+=AI

    tr3=0
    for a in range(1,3+1):
        for b in range(1,3+1):
            AI=np.trace(N1(a,b))*np.trace(N2(a,b))
            tr3+=AI

    sm=tr0-tr1-tr2+tr3
    amplitude=sm.real   #take the real part, the imagine part is zero
    return amplitude
###############################################################################################
'''                     calculated the differential cross section                           '''
###############################################################################################
"""
Generates random moments at an energy between 820.68 and 1003.06 Gev
and resolve the differential cross section numerically
"""
sp=[]  #mass center energy
SEp=[] #differential cross section 
Ang=[] #dispertion angle 
n=360  #random number control 
events=20000           #events
energy_gap0=energy[8]  #820.68 Gev
energy_gapn=energy[10] #1003.06 Gev
me=0.511e-3            #mass electron 

for i in range(events+1):  #cycle for an angle between 0 to 90 degree
    p1=random.randint(0,n) #generate random momentum 
    p2=random.randint(0,n)
    p3=random.randint(0,n)
    k1=random.randint(0,n)
    k2=random.randint(0,n)
    k3=random.randint(0,n)
    p0=(me**2+p1**2+p2**2+p3**2)**0.5
    k0=(k1**2+k2**2+k3**2)**0.5

    SM = amplitude(p1,p2,p3,k1,k2,k3,me)        #invariant amplitude 
    G = (pi/(alpha2_inv[10]*sqrt(2)*80.387**2)) #Fermi constant
    
    if ((p0+k0)>=energy_gap0):      #filter energy between 820.68 and 1003.06 Gev
        if ((p0+k0)<=energy_gapn):

            sp.append(p0+k0)
            s=(p0+k0)**2
            KP=k0*p0-k1*p1-k2*p2-k3*p3   
            CosAnlgle =(k1*p1+k2*p2+k3*p3) / (sqrt(k1**2+k2**2+k3**2)*sqrt(p1**2+p2**2+p3**2))
            Angle = acos(CosAnlgle)        #dispertion angle 
            #differential cross section 
            dSigma = (2*pi*sin(Angle)) * (G**2*(s-me**2)*SM)/(2*2*pi**2*16*(KP)*2*2*2*s) 
            SEp.append(dSigma/2.56819e-9)
            Ang.append(Angle*180/pi)

for i in range(events+1):  #cycle for an angle between 90 to 180 degree
    p1=random.randint(0,n)
    p2=random.randint(0,n)
    p3=random.randint(0,n)
    k1=-random.randint(0,n)
    k2=-random.randint(0,n)
    k3=-random.randint(0,n)
    p0=(me**2+p1**2+p2**2+p3**2)**0.5
    k0=(k1**2+k2**2+k3**2)**0.5

    SM = amplitude(p1,p2,p3,k1,k2,k3,me)
    G = (pi/(alpha2_inv[10]*sqrt(2)*80.387**2))
    
    if ((p0+k0)>=energy_gap0):
        if ((p0+k0)<=energy_gapn):

            sp.append(p0+k0)
            s=(p0+k0)**2
            KP=k0*p0-k1*p1-k2*p2-k3*p3
            CosAnlgle =(k1*p1+k2*p2+k3*p3) / (sqrt(k1**2+k2**2+k3**2)*sqrt(p1**2+p2**2+p3**2))
            Angle = acos(CosAnlgle)

            dSigma = (2*pi*sin(Angle)) * (G**2*(s-me**2)*SM)/(2*2*pi**2*16*(KP)*2*2*2*s) 
            SEp.append(dSigma/2.56819e-9)
            Ang.append(Angle*180/pi)
###############################################################################################
'''               make a curve that fits over the points of the dispersion                  '''
###############################################################################################
"""
As we use a Scipy function for regression,
we change to vectors from the numpy library.
"""
x=np.zeros(len(Ang)) 
y=np.zeros(len(SEp))
for i in range(0,len(Ang)): 
    x[i]=Ang[i]
    y[i]=SEp[i]
    
def f(x, a0,a1,a2,a3,a4,a5,a6,a7,a8): #define a polynomial for the adjusted curve
    return a0+a1*x+a2*x**2+a3*x**3+a4*x**4+a5*x**5+a6*x**6+a7*x**7+a8*x**8
a,pcov = curve_fit(f, x, y) #Find the polynomial coefficients that fit the data (x, y)

def regression(x): #regression of the data
    return a[0]+a[1]*x+a[2]*x**2+a[3]*x**3+a[4]*x**4+a[5]*x**5+a[6]*x**6+a[7]*x**7+a[8]*x**8

###############################################################################################
'''             Calculate total cross section by Monte Carlo method                         '''
###############################################################################################
"""
we introduce the adjustment as an array of numbers
"""
AngRegression=np.zeros(180)
SEpRegression=np.zeros(180)
for i in range(0,180):
    AngRegression[i]=i
    SEpRegression[i]=regression(i)

"""
Monte Carlo method to calculate the area under the curve
"""
dispertion_randomX=[]
dispertion_randomY=[] 
dispertion_BelowCurveX=[]
dispertion_BelowCurveY=[]
interaction = 50000
for i in range (interaction):
    x=180*random.random()
    y=max(SEpRegression+0.01)*random.random()
    if y<=regression(x):
        dispertion_BelowCurveX.append(x)
        dispertion_BelowCurveY.append(y)

    else:
        dispertion_randomX.append(x)
        dispertion_randomY.append(y)


areaMontecarlo=((len(dispertion_BelowCurveY))/(len(dispertion_BelowCurveY)+len(dispertion_randomY)))*(180*max(SEpRegression+0.01))*(pi/180)
areaMontecarlo #total cross section
